# Remote data access using pandas

The pandas library enables access to data displayed on websites using the `read_html()` function and access to the API endpoints of various data providers through the related `pandas-datareader` library.

In [1]:
import os
import pandas_datareader.data as web
from datetime import datetime
from pprint import pprint

## Download html table with SP500 constituents

The download of the content of one or more html tables works as follows, for instance for the constituents of the S&P500 index from Wikipedia

In [2]:
sp_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500_constituents = pd.read_html(sp_url, header=0)[0]

In [3]:
sp500_constituents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 9 columns):
Security                 505 non-null object
Symbol                   505 non-null object
SEC filings              505 non-null object
GICS Sector              505 non-null object
GICS Sub Industry        505 non-null object
Headquarters Location    505 non-null object
Date first added         402 non-null object
CIK                      505 non-null int64
Founded                  172 non-null object
dtypes: int64(1), object(8)
memory usage: 35.6+ KB


In [4]:
sp500_constituents.head()

,Security,Symbol,SEC filings,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded
0,3M Company,MMM,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,66740,1902
1,Abbott Laboratories,ABT,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,AbbVie Inc.,ABBV,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABIOMED Inc,ABMD,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,Accenture plc,ACN,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


## pandas-datareader for Market Data

`pandas` used to facilitate access to data providers' APIs directly, but this functionality has moved to the related pandas-datareader library. The stability of the APIs varies with provider policies, and as of June 2o18 at version 0.7, the following sources are available

See [documentation](https://pandas-datareader.readthedocs.io/en/latest/); functionality frequently changes as underlying provider APIs evolve.

### Yahoo Finance

In [5]:
start = '2014'
end = datetime(2017, 5, 24)

yahoo= web.DataReader('FB', 'yahoo', start=start, end=end)
yahoo.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 856 entries, 2014-01-02 to 2017-05-25
Data columns (total 6 columns):
High         856 non-null float64
Low          856 non-null float64
Open         856 non-null float64
Close        856 non-null float64
Volume       856 non-null int64
Adj Close    856 non-null float64
dtypes: float64(5), int64(1)
memory usage: 46.8 KB


### IEX

IEX is an alternative exchange started in response to the HFT controversy and portrayed in Michael Lewis' controversial Flash Boys. It aims to slow down the speed of trading to create a more level playing field and has been growing rapidly since launch in 2016 while still small with a market share of around 2.5% in June 2018.

In [6]:
start = datetime(2015, 2, 9)
# end = datetime(2017, 5, 24)

iex = web.DataReader('FB', 'iex', start)
iex.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1054 entries, 2015-02-09 to 2019-04-16
Data columns (total 5 columns):
open      1054 non-null float64
high      1054 non-null float64
low       1054 non-null float64
close     1054 non-null float64
volume    1054 non-null int64
dtypes: float64(4), int64(1)
memory usage: 49.4+ KB


In [7]:
iex.tail()

,open,high,low,close,volume
date,,,,,
2019-04-10,178.18,178.79,176.54,177.82,11701479
2019-04-11,178.24,178.40,177.00,177.51,8070967
2019-04-12,178.00,179.63,177.95,179.10,12329812
2019-04-15,178.50,180.50,176.87,179.65,10834762
2019-04-16,179.00,180.17,178.30,178.87,11215193


#### Book Data

In addition to historical EOD price and volume data, IEX provides real-time depth of book quotations that offer an aggregated size of orders by price and side. This service also includes last trade price and size information.

DEEP is used to receive real-time depth of book quotations direct from IEX. The depth of book quotations received via DEEP provide an aggregated size of resting displayed orders at a price and side, and do not indicate the size or number of individual orders at any price level. Non-displayed orders and non-displayed portions of reserve orders are not represented in DEEP.

DEEP also provides last trade price and size information. Trades resulting from either displayed or non-displayed orders matching on IEX will be reported. Routed executions will not be reported.

Only works on trading days.

In [8]:
book = web.get_iex_book('AAPL')

In [9]:
list(book.keys())

['symbol',
 'marketPercent',
 'volume',
 'lastSalePrice',
 'lastSaleSize',
 'lastSaleTime',
 'lastUpdated',
 'bids',
 'asks',
 'systemEvent',
 'tradingStatus',
 'opHaltStatus',
 'ssrStatus',
 'securityEvent',
 'trades',
 'tradeBreaks']

In [10]:
orders = pd.concat([pd.DataFrame(book[side]).assign(side=side) for side in ['bids', 'asks']])
orders.head()

,side


In [11]:
for key in book.keys():
    try:
        print(f'\n{key}')
        print(pd.DataFrame(book[key]))
    except:
        print(book[key])


symbol
AAPL

marketPercent
0.03324

volume
977659

lastSalePrice
203.19

lastSaleSize
3

lastSaleTime
1555531318248

lastUpdated
1555532174025

bids
Empty DataFrame
Columns: []
Index: []

asks
Empty DataFrame
Columns: []
Index: []

systemEvent
{'systemEvent': 'C', 'timestamp': 1555535400001}

tradingStatus
{'status': 'T', 'reason': '    ', 'timestamp': 1555500532036}

opHaltStatus
{'isHalted': False, 'timestamp': 1555500532036}

ssrStatus
{'isSSR': False, 'detail': ' ', 'timestamp': 1555500532036}

securityEvent
{'securityEvent': 'MarketClose', 'timestamp': 1555531200000}

trades
    isISO  isOddLot  isOutsideRegularHours  isSinglePriceCross  \
0    True      True                   True               False   
1   False     False                  False               False   
2    True     False                  False               False   
3   False     False                  False               False   
4    True     False                  False               False   
5   False     Fa

In [12]:
pd.DataFrame(book['trades']).head()

,isISO,isOddLot,isOutsideRegularHours,isSinglePriceCross,isTradeThroughExempt,price,size,timestamp,tradeId
0,True,True,True,False,False,203.190,3,1555531318248,891604355
1,False,False,False,False,False,203.200,100,1555531197248,890272160
2,True,False,False,False,False,203.140,100,1555531195857,889977488
3,False,False,False,False,False,203.155,100,1555531195465,889859713
4,True,False,False,False,False,203.140,100,1555531195292,889808657


### Quandl

In [13]:
symbol = 'FB.US'

quandl = web.DataReader(symbol, 'quandl', '2015-01-01')
quandl.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 813 entries, 2018-03-27 to 2015-01-02
Data columns (total 12 columns):
Open          813 non-null float64
High          813 non-null float64
Low           813 non-null float64
Close         813 non-null float64
Volume        813 non-null float64
ExDividend    813 non-null float64
SplitRatio    813 non-null float64
AdjOpen       813 non-null float64
AdjHigh       813 non-null float64
AdjLow        813 non-null float64
AdjClose      813 non-null float64
AdjVolume     813 non-null float64
dtypes: float64(12)
memory usage: 82.6 KB


### FRED

In [14]:
start = datetime(2010, 1, 1)

end = datetime(2013, 1, 27)

gdp = web.DataReader('GDP', 'fred', start, end)

gdp.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 13 entries, 2010-01-01 to 2013-01-01
Data columns (total 1 columns):
GDP    13 non-null float64
dtypes: float64(1)
memory usage: 208.0 bytes


In [15]:
inflation = web.DataReader(['CPIAUCSL', 'CPILFESL'], 'fred', start, end)
inflation.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 37 entries, 2010-01-01 to 2013-01-01
Freq: MS
Data columns (total 2 columns):
CPIAUCSL    37 non-null float64
CPILFESL    37 non-null float64
dtypes: float64(2)
memory usage: 888.0 bytes


### Fama/French

In [16]:
from pandas_datareader.famafrench import get_available_datasets
get_available_datasets()

['F-F_Research_Data_Factors',
 'F-F_Research_Data_Factors_weekly',
 'F-F_Research_Data_Factors_daily',
 'F-F_Research_Data_5_Factors_2x3',
 'F-F_Research_Data_5_Factors_2x3_daily',
 'Portfolios_Formed_on_ME',
 'Portfolios_Formed_on_ME_Wout_Div',
 'Portfolios_Formed_on_ME_Daily',
 'Portfolios_Formed_on_BE-ME',
 'Portfolios_Formed_on_BE-ME_Wout_Div',
 'Portfolios_Formed_on_BE-ME_Daily',
 'Portfolios_Formed_on_OP',
 'Portfolios_Formed_on_OP_Wout_Div',
 'Portfolios_Formed_on_INV',
 'Portfolios_Formed_on_INV_Wout_Div',
 '6_Portfolios_2x3',
 '6_Portfolios_2x3_Wout_Div',
 '6_Portfolios_2x3_weekly',
 '6_Portfolios_2x3_daily',
 '25_Portfolios_5x5',
 '25_Portfolios_5x5_Wout_Div',
 '25_Portfolios_5x5_Daily',
 '100_Portfolios_10x10',
 '100_Portfolios_10x10_Wout_Div',
 '100_Portfolios_10x10_Daily',
 '6_Portfolios_ME_OP_2x3',
 '6_Portfolios_ME_OP_2x3_Wout_Div',
 '6_Portfolios_ME_OP_2x3_daily',
 '25_Portfolios_ME_OP_5x5',
 '25_Portfolios_ME_OP_5x5_Wout_Div',
 '25_Portfolios_ME_OP_5x5_daily',
 '100_Po

In [17]:
ds = web.DataReader('5_Industry_Portfolios', 'famafrench')
print(ds['DESCR'])

5 Industry Portfolios
---------------------

This file was created by CMPT_IND_RETS using the 201901 CRSP database. It contains value- and equal-weighted returns for 5 industry portfolios. The portfolios are constructed at the end of June. The annual returns are from January to December. Missing data are indicated by -99.99 or -999. Copyright 2019 Kenneth R. French

  0 : Average Value Weighted Returns -- Monthly (109 rows x 5 cols)
  1 : Average Equal Weighted Returns -- Monthly (109 rows x 5 cols)
  2 : Average Value Weighted Returns -- Annual (9 rows x 5 cols)
  3 : Average Equal Weighted Returns -- Annual (9 rows x 5 cols)
  4 : Number of Firms in Portfolios (109 rows x 5 cols)
  5 : Average Firm Size (109 rows x 5 cols)
  6 : Sum of BE / Sum of ME (9 rows x 5 cols)
  7 : Value-Weighted Average of BE/ME (9 rows x 5 cols)


### World Bank

In [16]:
from pandas_datareader import wb
gdp_variables = wb.search('gdp.*capita.*const')
gdp_variables.head()

,id,name,source,sourceNote,sourceOrganization,topics,unit
646,6.0.GDPpc_constant,"GDP per capita, PPP (constant 2011 internation...",LAC Equity Lab,GDP per capita based on purchasing power parit...,b'World Development Indicators (World Bank)',Economy & Growth,
9108,NY.GDP.PCAP.KD,GDP per capita (constant 2010 US$),World Development Indicators,GDP per capita is gross domestic product divid...,"b'World Bank national accounts data, and OECD ...",Economy & Growth,
9110,NY.GDP.PCAP.KN,GDP per capita (constant LCU),World Development Indicators,GDP per capita is gross domestic product divid...,"b'World Bank national accounts data, and OECD ...",Economy & Growth,
9112,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2011 internation...",World Development Indicators,GDP per capita based on purchasing power parit...,"b'World Bank, International Comparison Program...",Economy & Growth,
9113,NY.GDP.PCAP.PP.KD.87,"GDP per capita, PPP (constant 1987 internation...",WDI Database Archives,,b'',,


In [18]:
wb_data = wb.download(indicator='NY.GDP.PCAP.KD', 
                      country=['US', 'CA', 'MX'], 
                      start=1990, 
                      end=2019)
wb_data.head()

NY.GDP.PCAP.KD
country year                
Canada  2018             NaN
        2017    51315.888975
        2016    50407.341330
        2015    50303.836848
        2014    50221.841982

### OECD

In [20]:
df = web.DataReader('TUD', 'oecd', end='2015')
df[['Japan', 'United States']]

Country            Japan                                                       \
Frequency         Annual                                                        
Source       Survey data                                                        
Series     Union members            Trade union  density            Employees   
Measure        Thousands Percentage            Thousands Percentage Thousands   
Year                                                                            
2010-01-01           NaN        NaN                  NaN        NaN       NaN   
2011-01-01           NaN        NaN                  NaN        NaN       NaN   
2012-01-01           NaN        NaN                  NaN        NaN       NaN   
2013-01-01           NaN        NaN                  NaN        NaN       NaN   
2014-01-01           NaN        NaN                  NaN        NaN       NaN   
2015-01-01           NaN        NaN                  NaN        NaN       NaN   

Country                                                                    \
Frequency                                                                   
Source                Administrative data                                   
Series                      Union members            Trade union  density   
Measure    Percentage           Thousands Percentage            Thousands   
Year                                                                        
2010-01-01        NaN           12417.527        NaN                  NaN   
2011-01-01        NaN           12271.909        NaN                  NaN   
2012-01-01        NaN           12227.073        NaN                  NaN   
2013-01-01        NaN           12396.592        NaN                  NaN   
2014-01-01        NaN            7661.568        NaN                  NaN   
2015-01-01        NaN            8359.876        NaN                  NaN   

Country                ...        United States                       \
Frequency              ...               Annual                        
Source                 ...          Survey data                        
Series                 ... Trade union  density            Employees   
Measure    Percentage  ...            Thousands Percentage Thousands   
Year                   ...                                             
2010-01-01  28.871256  ...                  NaN  17.448617   97406.0   
2011-01-01  27.589723  ...                  NaN  16.516118  102403.0   
2012-01-01  25.899329  ...                  NaN  15.861734  106924.0   
2013-01-01  24.489514  ...                  NaN  15.510448  107102.0   
2014-01-01  32.164433  ...                  NaN        NaN       NaN   
2015-01-01  34.516416  ...                  NaN        NaN       NaN   

Country                                                                    \
Frequency                                                                   
Source                Administrative data                                   
Series                      Union members            Trade union  density   
Measure    Percentage           Thousands Percentage            Thousands   
Year                                                                        
2010-01-01        NaN                 NaN        NaN                  NaN   
2011-01-01        NaN                 NaN        NaN                  NaN   
2012-01-01        NaN                 NaN        NaN                  NaN   
2013-01-01        NaN                 NaN        NaN                  NaN   
2014-01-01        NaN             17049.0        NaN                  NaN   
2015-01-01        NaN             16303.0        NaN                  NaN   

Country                                     
Frequency                                   
Source                                      
Series                Employees             
Measure    Percentage Thousands Percentage  
Year                                        
2010-01-01        NaN       NaN        NaN

### EuroStat

In [21]:
df = web.DataReader('tran_sf_railac', 'eurostat')

In [22]:
df.head()

ACCIDENT    Collisions of trains, including collisions with obstacles within the clearance gauge  \
UNIT                                                                                      Number   
GEO                                                                                      Austria   
FREQ                                                                                      Annual   
TIME_PERIOD                                                                                        
2010-01-01                                                 3.0                                     
2011-01-01                                                 2.0                                     
2012-01-01                                                 1.0                                     
2013-01-01                                                 4.0                                     
2014-01-01                                                 1.0                                     

ACCIDENT                                                         \
UNIT                                                              
GEO         Belgium Bulgaria Switzerland Channel Tunnel Czechia   
FREQ         Annual   Annual      Annual         Annual  Annual   
TIME_PERIOD                                                       
2010-01-01      5.0      2.0         5.0            0.0     3.0   
2011-01-01      0.0      0.0         4.0            0.0     6.0   
2012-01-01      3.0      3.0         4.0            0.0     6.0   
2013-01-01      1.0      2.0         6.0            0.0     5.0   
2014-01-01      3.0      4.0         0.0            0.0    13.0   

ACCIDENT                                                                      \
UNIT                                                                           
GEO         Germany (until 1990 former territory of the FRG) Denmark Estonia   
FREQ                                                  Annual  Annual  Annual   
TIME_PERIOD                                                                    
2010-01-01                                              13.0     0.0     1.0   
2011-01-01                                              18.0     1.0     0.0   
2012-01-01                                              23.0     1.0     3.0   
2013-01-01                                              29.0     0.0     0.0   
2014-01-01                                              32.0     0.0     0.0   

ACCIDENT            ...     Unknown                                        \
UNIT                ...      Number                                         
GEO         Greece  ... Netherlands Norway Poland Portugal Romania Sweden   
FREQ        Annual  ...      Annual Annual Annual   Annual  Annual Annual   
TIME_PERIOD         ...                                                     
2010-01-01     4.0  ...         NaN    NaN    NaN      NaN     NaN    NaN   
2011-01-01     1.0  ...         NaN    NaN    NaN      NaN     NaN    NaN   
2012-01-01     2.0  ...         NaN    NaN    NaN      NaN     NaN    NaN   
2013-01-01     2.0  ...         NaN    NaN    NaN      NaN     NaN    NaN   
2014-01-01     1.0  ...         NaN    NaN    NaN      NaN     NaN    NaN   

ACCIDENT                                             
UNIT                                                 
GEO         Slovenia Slovakia Turkey United Kingdom  
FREQ          Annual   Annual Annual         Annual  
TIME_PERIOD                                          
2010-01-01       NaN      NaN    0.0            NaN  
2011-01-01       NaN      NaN    0.0            NaN  
2012-01-01       NaN      NaN    0.0            NaN  
2013-01-01       NaN      NaN    0.0            NaN  
2014-01-01       NaN      NaN    0.0            NaN  

[5 rows x 264 columns]



### Stooq

Google finance stopped providing common index data download. The Stooq site had this data for download for a while but is currently broken, awaiting release of [fix](https://github.com/pydata/pandas-datareader/issues/594)

In [13]:
index_url = 'https://stooq.com/t/'
ix = pd.read_html(index_url)
len(ix)

46

In [14]:
f = web.DataReader('^SPX', 'stooq', start='20000101')
f.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Empty DataFrame

In [15]:
f.head()

""
No data


### NASDAQ Symbols

In [23]:
from pandas_datareader.nasdaq_trader import get_nasdaq_symbols
symbols = get_nasdaq_symbols()
symbols.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8701 entries, A to ZYXI
Data columns (total 11 columns):
Nasdaq Traded       8701 non-null bool
Security Name       8701 non-null object
Listing Exchange    8701 non-null category
Market Category     8701 non-null object
ETF                 8701 non-null bool
Round Lot Size      8701 non-null float64
Test Issue          8701 non-null bool
Financial Status    3411 non-null category
CQS Symbol          5290 non-null object
NASDAQ Symbol       8701 non-null object
NextShares          8701 non-null bool
dtypes: bool(4), category(2), float64(1), object(4)
memory usage: 459.2+ KB


In [24]:
url = 'https://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NASDAQ'
res = pd.read_html(url)
len(res)

4

In [25]:
for r in res:
    print(r.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 2 columns):
0    1 non-null object
1    1 non-null object
dtypes: object(2)
memory usage: 96.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 6 columns):
Name          101 non-null object
Symbol        51 non-null object
Market Cap    47 non-null object
Country       51 non-null object
IPO Year      28 non-null object
Subsector     51 non-null object
dtypes: object(6)
memory usage: 4.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 1 columns):
0    1 non-null object
dtypes: object(1)
memory usage: 88.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 1 columns):
0    1 non-null object
dtypes: object(1)
memory usage: 88.0+ bytes
None


### Tiingo

Requires [signing up](https://api.tiingo.com/) and storing API key in environment

In [26]:
df = web.get_data_tiingo('GOOG', api_key=os.getenv('TIINGO_API_KEY'))

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1244 entries, (GOOG, 2014-03-27 00:00:00) to (GOOG, 2019-03-06 00:00:00)
Data columns (total 12 columns):
adjClose       1244 non-null float64
adjHigh        1244 non-null float64
adjLow         1244 non-null float64
adjOpen        1244 non-null float64
adjVolume      1244 non-null int64
close          1244 non-null float64
divCash        1244 non-null float64
high           1244 non-null float64
low            1244 non-null float64
open           1244 non-null float64
splitFactor    1244 non-null float64
volume         1244 non-null int64
dtypes: float64(10), int64(2)
memory usage: 130.1+ KB
